Use the same preprocessing as propublica, and try to run my own algorithm: newAlgGeneral

For false positive/ false negaitve error rate


From r"../../InputData/Compas_ProPublica/compas-analysis-master/truth_tables.py", we know:

In data file r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed.csv"

For self.score in ["Low", "Medium", "High"]:

self.__rows[0]['is_recid'] == "1" and self.lifetime <= 730
or
i.lifetime > 730


In [1]:
# %load_ext autoreload
# %autoreload 2

# My own preprocessing

In [6]:
import pandas as pd

from truth_tables import PeekyReader, Person, table, is_race, count, vtable, hightable, vhightable
from csv import DictReader

rows = []
rindex = 0
pop = []


## select valid rows:
with open(r"../../InputData/COMPAS_ProPublica/compas-analysis-master/cox-parsed.csv") as f:
    reader = PeekyReader(DictReader(f))
    try:
        while True:
            p = Person(reader)
            if p.valid and p.score_valid:
                if (p.recidivist == True and p.lifetime <= 730) or p.lifetime > 730:
                    pop.append(p)
                    rows.append(rindex)
            rindex += 1
    except StopIteration:
        pass

len(rows)
# These rows are valid for FP, FN


7214

In [7]:
cox_parsed = pd.read_csv(r"../../InputData/COMPAS_ProPublica/compas-analysis-master/cox-parsed.csv")
print(len(cox_parsed))
cox_parsed[:4]


13419


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0
1,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,334,961,0
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0


In [103]:
# valid rows for experiment

cox_parsed_people = cox_parsed.loc[rows, : ]
cox_parsed_people.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed_7214rows.csv", index=False)

In [18]:
cox_parsed_people = pd.read_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed_7214rows.csv")
len(cox_parsed_people)


7214

## find TP, FP, TN, FN

In [21]:
# TP

TProws = []
FProws = []
TNrows = []
FNrows = []
rindex = 0
pop_rindex = []
num_recid = 0
num_surv = 0
people_rindex = []
recid_rindex = []
surv_rindex = []
## select valid rows:
with open("../../InputData/COMPAS_ProPublica/compas-analysis-master/cox-parsed.csv") as f:
    reader = PeekyReader(DictReader(f))
    try:
        while True:
            p = Person(reader)
            if p.valid: # people
                people_rindex.append(rindex)
                if p.score_valid:
                    if (p.recidivist == True and p.lifetime <= 730) or p.lifetime > 730: # pop
                        pop_rindex.append(rindex)
                        if p.recidivist == True and p.lifetime <= 730: # recid
                            recid_rindex.append(rindex)
                            if p.low:
                                FNrows.append(rindex)
                            elif p.high:
                                TProws.append(rindex)
                        else: # survive
                            surv_rindex.append(rindex)
                            if p.low:
                                TNrows.append(rindex)
                            else:
                                FProws.append(rindex)
            rindex += p.num_rows
    except StopIteration:
        pass

print(rindex)
print("len(people_rindex)={}".format(len(people_rindex)))
print("len(pop_rindex)={}".format(len(pop_rindex)))
print("len(recid_rindex)={}".format(len(recid_rindex)))
print("len(surv_rindex)={}".format(len(surv_rindex)))

print("num_recid, num_surv", len(TProws) + len(FNrows), len(TNrows) + len(FProws), )
print(len(FNrows), len(TProws), len(FProws), len(TNrows))

print("False posirive rate: {}".format(len(FProws)/(len(FProws) + len(TNrows))))
print("False negative rate: {}".format(len(FNrows)/(len(FNrows) + len(TProws))))


13419
len(people_rindex)=7225
len(pop_rindex)=7214
len(recid_rindex)=3251
len(surv_rindex)=3963
num_recid, num_surv 3251 3963
1216 2035 1282 2681
False posirive rate: 0.32349230381024474
False negative rate: 0.3740387573054445


In [124]:
cox_parsed = pd.read_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed.csv")

print(len(cox_parsed))
print(cox_parsed[12:15])

cox_parsed[:4]



13419
    id               name      first       last compas_screening_date     sex  \
12  10   elizabeth thieme  elizabeth     thieme            2014-03-16  Female   
13  11  darrell blackburn    darrell  blackburn            2013-12-09    Male   
14  11  darrell blackburn    darrell  blackburn            2013-12-09    Male   

           dob  age       age_cat       race  ...  v_type_of_assessment  \
12  1976-06-03   39       25 - 45  Caucasian  ...      Risk of Violence   
13  1995-11-16   20  Less than 25  Caucasian  ...      Risk of Violence   
14  1995-11-16   20  Less than 25  Caucasian  ...      Risk of Violence   

    v_decile_score  v_score_text  v_screening_date  in_custody  out_custody  \
12               1           Low        2014-03-16  2014-03-15   2014-03-18   
13               9          High        2013-12-09  2014-08-06   2020-01-01   
14               9          High        2013-12-09  2015-04-07   2015-04-20   

   priors_count.1 start  end event  
12            

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0
1,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,334,961,0
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0


In [26]:

print(len(FProws))
print(FProws[:5])



1282
[5, 28, 64, 71, 101]


In [11]:

cox_parsed_people[:2]




,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,predicted,ground_truth
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,334,961,0,0,0


In [22]:



predicted = [0] * len(cox_parsed_people)
ground_truth = [0] * len(cox_parsed_people)
i = -1

for j in range(len(cox_parsed)):
    i += 1
    if j in TProws:
        predicted[i] = 1
        ground_truth[i] = 1
    elif j in TNrows:
        predicted[i] = 0
        ground_truth[i] = 0
    elif j in FProws:
        predicted[i] = 1
        ground_truth[i] = 0
    elif j in FNrows:
        predicted[i] = 0
        ground_truth[i] = 1
    else:
        i -= 1
        # raise Exception("{} found nowhere".format(i))

print("i = {}".format(i))
print(predicted, ground_truth)



num = 7213
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0

In [23]:


### add prediction and truth columns to cox_parsed
cox_parsed_people['predicted'] = predicted
cox_parsed_people['ground_truth'] = ground_truth




In [24]:

cox_parsed_people[:3]



,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,predicted,ground_truth
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,334,961,0,0,1
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,0,1


In [29]:


### add prediction and truth columns to cox_parsed
cox_parsed['predicted'] = [0] * len(cox_parsed)
cox_parsed['ground_truth'] = [0] * len(cox_parsed)
cox_parsed[:4]

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,predicted,ground_truth
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,334,961,0,0,0
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,0,0
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,0,0


In [30]:

cox_parsed.loc[TProws, 'predicted'] = 1
cox_parsed.loc[TProws, 'ground_truth'] = 1
cox_parsed.loc[TNrows, 'predicted'] = 0
cox_parsed.loc[TNrows, 'ground_truth'] = 0
cox_parsed.loc[FProws, 'predicted'] = 1
cox_parsed.loc[FProws, 'ground_truth'] = 0
cox_parsed.loc[FNrows, 'predicted'] = 0
cox_parsed.loc[FNrows, 'ground_truth'] = 1
cox_parsed[:10]


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,predicted,ground_truth
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,334,961,0,0,0
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,0,1
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,0,1
4,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,Low,2013-04-14,2013-06-16,2013-06-16,4,63,64,1,0,0
5,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,Medium,2013-01-13,NaN,NaN,1,0,1174,0,1,0
6,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,Low,2013-03-26,NaN,NaN,2,0,1102,0,0,0
7,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0,0
8,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1,1
9,9,steven stewart,steven,stewart,2013-08-30,Male,1973-02-25,43,25 - 45,Other,...,Low,2013-08-30,2014-05-22,2014-06-03,3,0,265,0,0,0


In [31]:

cox_parsed_people = cox_parsed.loc[pop_rindex, :]
print(len(cox_parsed_people))



7214


In [32]:


cox_parsed_people.to_csv(r"../../InputData/COMPAS_ProPublica/compas-analysis-master/cox-parsed/cox-parsed_7214rows_with_labels.csv", index=False)







### finish adding predicted and ground truth rows


In [125]:

print(TProws[:5])

[8, 21, 30, 32, 33]


In [126]:
TPdata = cox_parsed.loc[ TProws, : ]
print(len(TPdata))

print(TPdata[:5])

TPdata.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed-TP.csv")


2035
    id                 name       first      last compas_screening_date   sex  \
8    8        edward riddle      edward    riddle            2014-02-19  Male   
21  15       ellyaher lanza    ellyaher     lanza            2013-10-03  Male   
30  23         neil heckart        neil   heckart            2013-11-17  Male   
32  25      columbus wilson    columbus    wilson            2014-05-02  Male   
33  26  vandivuiet williams  vandivuiet  williams            2013-04-18  Male   

           dob  age          age_cat              race  ...  \
8   1974-07-23   41          25 - 45         Caucasian  ...   
21  1992-08-18   23     Less than 25  African-American  ...   
30  1984-12-24   31          25 - 45         Caucasian  ...   
32  1951-06-28   64  Greater than 45  African-American  ...   
33  1994-11-29   21     Less than 25  African-American  ...   

    v_type_of_assessment  v_decile_score  v_score_text  v_screening_date  \
8       Risk of Violence               2           Lo

In [127]:
FPdata = cox_parsed.loc[ FProws, : ]
print(len(FPdata))

FPdata.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed-FP.csv")


1282


In [128]:
TNdata = cox_parsed.loc[ TNrows, : ]
print(len(TNdata))

TNdata.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed-TN.csv")


2681


In [129]:
FNdata = cox_parsed.loc[ FNrows, : ]
print(len(FNdata))

FNdata.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed-FN.csv")


1216


In [146]:
Misrows = FNrows + FProws
Misrows[:4]


[2, 3, 19, 25]

In [147]:
mis_data = cox_parsed.loc[Misrows, : ]
print(len(mis_data))

2498


In [148]:
mis_data.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed_mis_cat.csv")


## categorize age, race, sex

In [132]:
cox = pd.read_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed.csv")


In [133]:
# replace string with numerics

cox['sex'].replace(to_replace=['Male', 'Female'], value=[0, 1], inplace=True)

cox['sex']

0        0
1        0
2        0
3        0
4        0
        ..
13414    1
13415    0
13416    1
13417    1
13418    1
Name: sex, Length: 13419, dtype: int64

In [134]:
cox['age_cat'].replace(to_replace=['Less than 25', '25 - 45', 'Greater than 45'], value=[0, 1, 2], inplace=True)
cox['age_cat']

0        2
1        2
2        1
3        0
4        0
        ..
13414    1
13415    2
13416    1
13417    0
13418    0
Name: age_cat, Length: 13419, dtype: int64

In [135]:
cox['race'].replace(to_replace=['African-American', 'Asian', 'Caucasian', 'Hispanic', 'Native American', 'Other'], value=[0, 1, 2, 3, 4, 5], inplace=True)
cox['race']

0        5
1        5
2        0
3        0
4        0
        ..
13414    0
13415    5
13416    0
13417    3
13418    3
Name: race, Length: 13419, dtype: int64

In [136]:
cox.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed-cat.csv")


In [137]:

TPdata = cox.loc[ TProws, : ]
print(len(TPdata))

print(TPdata[:8])

TPdata.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed-TP-cat.csv")

2035
    id                 name       first      last compas_screening_date  sex  \
8    8        edward riddle      edward    riddle            2014-02-19    0   
21  15       ellyaher lanza    ellyaher     lanza            2013-10-03    0   
30  23         neil heckart        neil   heckart            2013-11-17    0   
32  25      columbus wilson    columbus    wilson            2014-05-02    0   
33  26  vandivuiet williams  vandivuiet  williams            2013-04-18    0   
47  39           najee sapp       najee      sapp            2013-02-20    0   
49  40        victor moreno      victor    moreno            2014-10-24    0   
51  41       william pirkle     william    pirkle            2013-01-28    0   

           dob  age  age_cat  race  ...  v_type_of_assessment  v_decile_score  \
8   1974-07-23   41        1     2  ...      Risk of Violence               2   
21  1992-08-18   23        0     0  ...      Risk of Violence               4   
30  1984-12-24   31        1   

In [138]:
FPdata = cox.loc[ FProws, : ]
print(len(FPdata))

FPdata.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed-FP-cat.csv")


1282


In [139]:
TNdata = cox.loc[ TNrows, : ]
print(len(TNdata))

TNdata.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed-TN-cat.csv")


2681


In [140]:
FNdata = cox.loc[ FNrows, : ]
print(len(FNdata))

FNdata.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed-FN-cat.csv")



1216


In [141]:

popdata = cox.loc[pop_rindex, : ]
print(len(pop_rindex))

popdata.to_csv(r"../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed/cox-parsed_7214rows_cat.csv")





7214


This is the end of categorization


## Test data


In [99]:
print("num_recid, num_surv", len(TProws) + len(FNrows), len(TNrows) + len(FProws), )
print(len(FNrows), len(TProws), len(FProws), len(TNrows))

print("False posirive rate: {}".format(len(FProws)/(len(FProws) + len(TNrows))))
print("False negative rate: {}".format(len(FNrows)/(len(FNrows) + len(TProws))))


print( len(popdata[ popdata['race'] == 0 ]), len(popdata[ popdata['race'] == 1 ]) )

print(len(TPdata), len(FNdata))
print(len(FPdata[FPdata['race'] == 0]))

num_recid, num_surv 3251 3963
1216 2035 1282 2681
False posirive rate: 0.32349230381024474
False negative rate: 0.3740387573054445
3715 35
2035 1216
655


# preprocessing from ProPublica


## Directions of the Racial Bias

The above analysis shows that the Compas algorithm does overpredict African-American defendant's future recidivism, but we haven't yet explored the direction of the bias. We can discover fine differences in overprediction and underprediction by comparing Compas scores across racial lines.

In [49]:
from truth_tables import PeekyReader, Person, table, is_race, count, vtable, hightable, vhightable
from csv import DictReader

rows = []
rindex = 0
people = []
with open("../../InputData/Compas_ProPublica/compas-analysis-master/cox-parsed.csv") as f:
    reader = PeekyReader(DictReader(f))
    try:
        while True:
            p = Person(reader)
            if p.valid:
                people.append(p)
                rows.append(rindex)
            rindex += 1
    except StopIteration:
        pass

pop = list(filter(lambda i: ((i.recidivist == True and i.lifetime <= 730) or
                              i.lifetime > 730), list(filter(lambda x: x.score_valid, people))))
recid = list(filter(lambda i: i.recidivist == True and i.lifetime <= 730, pop))
rset = set(recid)
surv = [i for i in pop if i not in rset]

print(len(rows))
print("length of pop, recid, rset, surv:", len(pop), len(recid), len(rset), len(surv))



7225
length of pop, recid, rset, surv: 7214 3251 3251 3963


In [48]:
print("All defendants")

# recid: recidivism, surv: not recid
table(list(recid), list(surv))

All defendants
           	Low	High
Survived   	2681	1282	0.55
Recidivated	1216	2035	0.45
Total: 7214.00
False positive rate: 32.35
False negative rate: 37.40
Specificity: 0.68
Sensitivity: 0.63
Prevalence: 0.45
PPV: 0.61
NPV: 0.69
LR+: 1.94
LR-: 0.55


In [ ]:
print("Total pop: %i" % (2681 + 1282 + 1216 + 2035))

# FP/(FP+TN)
print("False positive rate: {}".format(1282/(1282+2681)))

# FN/(FN+TP)
print("False negative rate: {}".format(1216/(1216+2035)))


In [ ]:
import statistics
print("Average followup time %.2f (sd %.2f)" % (statistics.mean(map(lambda i: i.lifetime, pop)),
                                                statistics.stdev(map(lambda i: i.lifetime, pop))))
print("Median followup time %i" % (statistics.median(map(lambda i: i.lifetime, pop))))

Overall, the false positive rate is 32.35%.

In [ ]:
print("Black defendants")
is_afam = is_race("African-American")
table(list(filter(is_afam, recid)), list(filter(is_afam, surv)))

That number is higher for African Americans at 44.85%.

In [ ]:
print("White defendants")
is_white = is_race("Caucasian")
table(list(filter(is_white, recid)), list(filter(is_white, surv)))

And lower for whites at 23.45%.

In [ ]:
44.85 / 23.45

Which means under COMPAS black defendants are 91% more likely to get a higher score and not go on to commit more crimes than white defendants after two year.
